In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("/home/vmeshchaninov/DiffusionTextGeneration-cond-ca/")

In [3]:
pred = [
    "The player stood in the field looking at the batter.",
    "The coach stands along the field, looking at the goalkeeper.",
    "I stood and looked across the field, peacefully.",
    "I stood and looked across the field.",
    "Someone stands, looking around the empty field."
]

In [9]:
import numpy as np
from estimation_utils.pymteval import BLEUScore

def bleu_score(predictions, references):
    scorer = BLEUScore(smoothing=1.0)
    scorer.reset()
    scorer.append(predictions, [references])
    return scorer.score()

def mbr(samples):

    score_dict = {}
    for idx_y in range(len(samples)):
        y = samples[idx_y]
        
        blue_lst = []
        for idx_x in range(len(samples)):
            if idx_x != idx_y:
                x = samples[idx_x]
                b = bleu_score(predictions=y, references=x)
                blue_lst.append(b)
        score_dict[idx_y] = np.array(blue_lst).mean()

    best_y = sorted(score_dict.items(), key=lambda item: item[1])[-1]

    return samples[best_y[0]]

In [10]:
mbr(pred)

[0.24737714147554013, 0.17033186037639278, 0.17033186037639278, 0.14323145079400493]
[0.24712442545253582, 0.2044800736021839, 0.14694106251955755, 0.1690062198556585]
[0.1705647399369684, 0.20311412297014433, 0.6989307622784944, 0.15851165692617156]
[0.16481400866629634, 0.13755608571892394, 0.6924626985290344, 0.20014292374951972]
[0.14211011212459493, 0.16449759298465816, 0.15881076016027915, 0.19960198807747329]


'I stood and looked across the field, peacefully.'

In [4]:
from transformers import AutoTokenizer

In [6]:
from estimation_utils.nmt_bleu import compute_bleu as bleu
tokenizer_mbert = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [12]:
references = [[tokenizer_mbert.tokenize(item)] for item in pred[:2]]
predictions = [tokenizer_mbert.tokenize(item) for item in pred[3:]]

In [13]:
results = bleu(reference_corpus=references, translation_corpus=predictions, max_order=4, smooth=True)

In [14]:
results

(0.10705967064287497,
 [0.6111111111111112, 0.1875, 0.07142857142857142, 0.08333333333333333],
 0.6624801353939264,
 0.7083333333333334,
 17,
 24)

In [23]:
from rouge_score import rouge_scorer, scoring

In [24]:
aggregator = scoring.BootstrapAggregator()

In [25]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
aggregator.add_scores(scores)

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
aggregator.add_scores(scores)

In [26]:
result = aggregator.aggregate()

In [27]:
result

{'rouge1': AggregateScore(low=Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765), mid=Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765), high=Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765)),
 'rouge2': AggregateScore(low=Score(precision=0.2857142857142857, recall=0.25, fmeasure=0.26666666666666666), mid=Score(precision=0.2857142857142857, recall=0.25, fmeasure=0.26666666666666666), high=Score(precision=0.2857142857142857, recall=0.25, fmeasure=0.26666666666666666)),
 'rougeL': AggregateScore(low=Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471), mid=Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471), high=Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471))}